In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# load in data
df_train = pd.read_csv('/home/tomruge/Schreibtisch/Data/Kaggle/playground-series-s4e3/train.csv')
df_test = pd.read_csv('/home/tomruge/Schreibtisch/Data/Kaggle/playground-series-s4e3/test.csv')


-> decision tree performs worse then logistic regression

In [13]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import optuna
from sklearn.metrics import roc_auc_score
class model_decision_tree:
    def __init__(self, train, test, target, drop = []):
        self.train = train
        self.test = test
        self.target = target
        self.drop = drop
    
    def fit(self, params = {}):
        # running fit multiple times for different validation and train sets to get a better estimate for the prediction
        predictions = []
        for i in range(5):
            # MultilabelStratifiedKFold splits 5 times, but the splits are not overlapping
            mskf = MultilabelStratifiedKFold(n_splits=3, shuffle=True, random_state=42)
            predictions_1_iteration = np.zeros((len(self.train),len(self.target)))
            #print('i: ', i)
            for fold, (train_index, val_index) in enumerate(mskf.split(self.train.drop(self.target, axis=1), self.train[self.target])):
                X_train, X_val = self.train.drop(self.target, axis=1).iloc[train_index], self.train.drop(self.target, axis=1).iloc[val_index]
                y_train, y_val = self.train[self.target].iloc[train_index], self.train[self.target].iloc[val_index]

                model = DecisionTreeClassifier(random_state = i+fold,**params)
                model.fit(X_train, y_train)
                predictions_1_iteration[val_index] = model.predict(X_val)  
                # print the accuracy of the model of the validation set of all the classes
                print('accuracy of the model of the validation set of all the classes: ', model.score(X_val, y_val))
            predictions.append(predictions_1_iteration)
        # get the mean of the predictions
        mean_predictions = np.mean(predictions, axis = 0)
        '''print('shape of mean_predictions: ', mean_predictions.shape)
        print('mean unique values: ', np.unique(mean_predictions))
        print('mean_predictions: ', mean_predictions)'''
        # get roc auc score
        integral_score = roc_auc_score(self.train[self.target], mean_predictions)
        return integral_score, mean_predictions   
     
    def prediction(self,params):
        predictions = []
        for i in range(10):
            mskf = MultilabelStratifiedKFold(n_splits=3, shuffle=True, random_state=42)
            for fold, (train_index, val_index) in enumerate(mskf.split(self.train.drop(self.target, axis=1), self.train[self.target])):
                X_train, X_val = self.train.drop(self.target, axis=1).iloc[train_index], self.train.drop(self.target, axis=1).iloc[val_index]
                y_train, y_val = self.train[self.target].iloc[train_index], self.train[self.target].iloc[val_index]
                model = DecisionTreeClassifier(random_state = i+fold,**params)
                model.fit(X_train, y_train)
                prediction = model.predict(self.test)
                predictions.append(prediction) 
        # predictions shape
        print('predictions shape: ', np.array(predictions).shape)           
        mean_predictions = np.mean(predictions, axis = 0)
        return mean_predictions
    
    def objective(self, trial) :
        params = {
            'max_depth': trial.suggest_int('max_depth', 1, 32),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32)
            #'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
        }
        score, _ = self.fit(params)
        return score
    
    def find_params(self):
        print('start finding best params')
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=20, n_jobs = -1)
        return study.best_params


       

In [4]:
# find best params
target_columns = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']
drop_columns = ['id']
model = model_decision_tree(df_train, df_test, target_columns, drop_columns)
best_params = model.find_params()

[I 2024-03-10 16:34:56,695] A new study created in memory with name: no-name-a900eecf-cecf-4ffa-8e44-88d94c4d9fb7


start finding best params


[I 2024-03-10 16:35:42,269] Trial 1 finished with value: 0.609843387009957 and parameters: {'max_depth': 4, 'min_samples_split': 31, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.609843387009957.
[I 2024-03-10 16:35:54,281] Trial 3 finished with value: 0.6754967334537779 and parameters: {'max_depth': 9, 'min_samples_split': 23, 'min_samples_leaf': 7}. Best is trial 3 with value: 0.6754967334537779.
[I 2024-03-10 16:36:00,696] Trial 6 finished with value: 0.6730756204225353 and parameters: {'max_depth': 16, 'min_samples_split': 19, 'min_samples_leaf': 23}. Best is trial 3 with value: 0.6754967334537779.
[I 2024-03-10 16:36:09,175] Trial 0 finished with value: 0.6730690468152047 and parameters: {'max_depth': 24, 'min_samples_split': 7, 'min_samples_leaf': 23}. Best is trial 3 with value: 0.6754967334537779.
[I 2024-03-10 16:36:10,414] Trial 5 finished with value: 0.6704426357058041 and parameters: {'max_depth': 25, 'min_samples_split': 21, 'min_samples_leaf': 24}. Best is trial 

In [14]:
target_columns = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']
drop_columns = ['id']
best_params = {'max_depth': 14, 'min_samples_split': 12, 'min_samples_leaf': 1}
# predict and save the predictions
model = model_decision_tree(df_train, df_test, target_columns, drop_columns)
predicted_values = model.fit(best_params)

accuracy of the model of the validation set of all the classes:  0.4382706414858748
accuracy of the model of the validation set of all the classes:  0.4399000936621917
accuracy of the model of the validation set of all the classes:  0.4512956603184515
accuracy of the model of the validation set of all the classes:  0.4402996722334946
accuracy of the model of the validation set of all the classes:  0.4419294411489229
accuracy of the model of the validation set of all the classes:  0.44989072744302216
accuracy of the model of the validation set of all the classes:  0.43936319650382394
accuracy of the model of the validation set of all the classes:  0.4378707461754605
accuracy of the model of the validation set of all the classes:  0.4516078676241024
accuracy of the model of the validation set of all the classes:  0.43670984860309037
accuracy of the model of the validation set of all the classes:  0.4388073680924134
accuracy of the model of the validation set of all the classes:  0.452388

In [38]:
target_columns = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']
drop_columns = ['id']
best_params = {'max_depth': 14, 'min_samples_split': 12, 'min_samples_leaf': 1}
# predict and save the predictions
model = model_decision_tree(df_train, df_test, target_columns, drop_columns)
predicted_values = model.prediction(best_params)

# column 1 must be the id column and the following columns must be the predictions, id column should be integer
predictions = pd.DataFrame(np.column_stack((df_test['id'].astype('Int32'), predicted_values)), columns = ['id', 'Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults'])
predictions.to_csv('/home/tomruge/Schreibtisch/Data/Kaggle/playground-series-s4e3/predictions.csv', index = False)


predictions shape:  (30, 12814, 7)
